In [1]:
from dataclasses import dataclass
from typing import Annotated
from typing_extensions import TypeAliasType

from pydantic import BeforeValidator, AfterValidator, WrapValidator, PlainValidator, BaseModel, PlainSerializer, WithJsonSchema

from pydantic_core import core_schema

@dataclass
class MyClass:
    value: int = 5
    unit: str = "wow"

In [2]:
def dumb_validator(value, info):
    return value

def dumb_ser(value):
    return str(value)

In [26]:
class MyGoodValidator:
    @classmethod
    def __get_pydantic_core_schema__(cls, source_type, handler):
        def moo_moo(value, info):
            print(f"{value=}")
            print(f"{info=}")
            print(f"{handler=}")
            return value

        duh_schema = core_schema.with_info_plain_validator_function(moo_moo)
        # duh_schema = core_schema.chain_schema(
        #     [
        #         core_schema.with_info_plain_validator_function(moo_moo),
        #         core_schema.str_schema(),
        #     ]
        # )

        duh_return = core_schema.json_or_python_schema(
            # json_schema=core_schema.str_schema(),
            python_schema=duh_schema,
#            serialization=core_schema.plain_serializer_function_ser_schema(lamba instance: str(instance))
        )
        return duh_return
        

In [27]:
MyClassType = TypeAliasType(
    'MyClassType',
    Annotated[
        MyClass,
        PlainValidator(dumb_validator),
        PlainSerializer(dumb_ser),
        WithJsonSchema(core_schema.str_schema())
    ]
)

class MyClassType2(MyClass):
    def 

In [28]:
MyClassType2 = TypeAliasType(
    'MyClassType2',
    Annotated[
        MyClass,
        MyGoodValidator()
    ]
)

In [29]:
class MyModel2(BaseModel):
    a: MyClassType2
    b: MyClassType2

TypeError: json_or_python_schema() missing 1 required positional argument: 'json_schema'

In [30]:
mm2 = MyModel2(a=MyClass(), b=MyClass())

value=MyClass(value=5, unit='wow')
info=ValidationInfo(config={'title': 'MyModel2'}, context=None, data={}, field_name=None)
handler=<pydantic._internal._schema_generation_shared.CallbackGetCoreSchemaHandler object at 0x1212c9e50>
value=MyClass(value=5, unit='wow')
info=ValidationInfo(config={'title': 'MyModel2'}, context=None, data={'a': MyClass(value=5, unit='wow')}, field_name=None)
handler=<pydantic._internal._schema_generation_shared.CallbackGetCoreSchemaHandler object at 0x1212c9e50>


In [8]:
mm2

MyModel2(a=MyClass(value=5, unit='wow'), b=MyClass(value=5, unit='wow'))

In [9]:
mm2.model_json_schema()

{'$defs': {'MyClassType2': {'type': 'string'}},
 'properties': {'a': {'$ref': '#/$defs/MyClassType2'},
  'b': {'$ref': '#/$defs/MyClassType2'}},
 'required': ['a', 'b'],
 'title': 'MyModel2',
 'type': 'object'}

In [10]:
class MyModel(BaseModel):
    a_thing: MyClassType
    b_thing: MyClassType

In [11]:
moo = MyModel(a_thing=5, b_thing=7)  # MyModel(a_thing=MyClassType, b_thing="hello")

In [12]:
moo.model_dump()

{'a_thing': '5', 'b_thing': '7'}

In [13]:
moo.model_dump_json()

'{"a_thing":"5","b_thing":"7"}'

In [14]:
moo.model_json_schema()

{'$defs': {'MyClassType': {'type': 'str'}},
 'properties': {'a_thing': {'$ref': '#/$defs/MyClassType'},
  'b_thing': {'$ref': '#/$defs/MyClassType'}},
 'required': ['a_thing', 'b_thing'],
 'title': 'MyModel',
 'type': 'object'}

In [15]:
class MyModelInt(BaseModel):
    a_thing: int
    b_thing: int

In [16]:
ioo = MyModelInt(a_thing=1, b_thing=2)

In [17]:
ioo.model_json_schema()

{'properties': {'a_thing': {'title': 'A Thing', 'type': 'integer'},
  'b_thing': {'title': 'B Thing', 'type': 'integer'}},
 'required': ['a_thing', 'b_thing'],
 'title': 'MyModelInt',
 'type': 'object'}

In [31]:
MyBooValidator = TypeAliasType(
    'MyBooValidator',
    Annotated[
        MyGoodValidator,
        WithJsonSchema(core_schema.str_schema())
    ]
)
    

In [32]:
MyClassType3 = TypeAliasType(
    'MyClassType3',
    Annotated[
        MyClass,
        MyBooValidator
    ]
)

In [33]:
class MyModel3(BaseModel):
    kanga: MyClassType3
    roo: MyClassType3

SchemaError: Invalid Schema:
definitions.definitions.0.definition-ref.ref
  Extra inputs are not permitted [type=extra_forbidden, input_value='__main__.MyClassType3:4909092624', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/extra_forbidden

In [23]:
mm3 = MyModel3(kanga="oo", roo="pp")

value='oo'
info=ValidationInfo(config={'title': 'MyModel3'}, context=None, data={}, field_name=None)
handler=<pydantic._internal._schema_generation_shared.CallbackGetCoreSchemaHandler object at 0x1230add10>
value='pp'
info=ValidationInfo(config={'title': 'MyModel3'}, context=None, data={'kanga': 'oo'}, field_name=None)
handler=<pydantic._internal._schema_generation_shared.CallbackGetCoreSchemaHandler object at 0x1230add10>


In [24]:
mm3.model_json_schema()

{'$defs': {'MyClassType2': {'type': 'string'}},
 'properties': {'kanga': {'$ref': '#/$defs/MyClassType2'},
  'roo': {'$ref': '#/$defs/MyClassType2'}},
 'required': ['kanga', 'roo'],
 'title': 'MyModel3',
 'type': 'object'}